In [1]:
from common.inputreader import InputReader, PuzzleWrapper

puzzle = PuzzleWrapper(year=2024, day=int("20"))

puzzle.header()

# Race Condition

[Open Website](https://adventofcode.com/2024/day/20)

In [2]:
# helper functions
def domain_from_input(input: InputReader) -> (list, list, list):
    matrix = input.matrix()

    # find start
    start = None
    for x, y, value in matrix:
        if value == 'S':
            start = (x, y)
            break

    # find end
    end = None
    for x, y, value in matrix:
        if value == 'E':
            end = (x, y)
            break

    return matrix, start, end


test_input, _, _ = domain_from_input(puzzle.example(0))
test_input.print()

###############
#...#...#.....#
#.#.#.#.#.###.#
#S#...#.#.#...#
#######.#.#.###
#######.#.#...#
#######.#.###.#
###..E#...#...#
###.#######.###
#...###...#...#
#.#####.#.###.#
#.#...#.#.#...#
#.#.#.#.#.#.###
#...#...#...###
###############


In [3]:
# test case (part 1)
from functools import cache
from common.matrix import Matrix, MatrixNavigator, Direction
from collections import deque

directions = [
    Direction.UP,
    Direction.DOWN,
    Direction.LEFT,
    Direction.RIGHT
]


@cache
def available_options(matrix: Matrix, point: tuple, with_cheat: bool) -> list:
    pointer = MatrixNavigator(matrix, point[0], point[1])
    options = []
    for direction in directions:
        ok, value = pointer.peek_value(direction)
        if ok:
            if with_cheat or value != '#':
                new_pointer = pointer.copy()
                new_pointer.move(direction)
                options.append(new_pointer.get_position())

    return options


@cache
def find_shortest_path(matrix: Matrix, start: tuple, end: tuple) -> list:
    queue = deque([(start, [start])])
    visited = set()
    visited.add(start)

    while queue:
        current_point, path = queue.popleft()
        if current_point == end:
            return path

        for next_point in available_options(matrix, current_point, False):
            if next_point not in visited:
                visited.add(next_point)
                queue.append((next_point, path + [next_point]))

    return []  # Return an empty list if no path is found


@cache
def find_shortest_path_count(matrix: Matrix, start: tuple, end: tuple) -> int:
    queue = deque([(start, 0)])
    visited = set()
    visited.add(start)

    while queue:
        current_point, path_length = queue.popleft()
        if current_point == end:
            return path_length

        for next_point in available_options(matrix, current_point, False):
            if next_point not in visited:
                visited.add(next_point)
                queue.append((next_point, path_length + 1))

    return -1  # Return -1 if no path is found


def part_1(reader: InputReader, min_savings: int, debug: bool) -> int:
    matrix, start, end = domain_from_input(reader)
    initial_solution = find_shortest_path(matrix, start, end)
    initial_solution_length = len(initial_solution)

    solutions = {}

    for point in initial_solution:
        new_starts = []
        steps = initial_solution.index(point)

        # find all "." within two steps
        for point_1 in available_options(matrix, point, True):
            for point_2 in available_options(matrix, point_1, False):
                new_starts.append(point_2)

        steps += 2
        for new_start in new_starts:
            # if there's a solution
            solution_length = find_shortest_path_count(matrix, new_start, end) + 1
            if solution_length > -1:
                new_total = solution_length + steps
                if new_total < initial_solution_length:
                    solutions[new_total] = solutions.get(new_total, 0) + 1

    total = 0

    if debug:
        matrix.print()
        print()

    for key in sorted(solutions.keys(), reverse=True):
        savings = initial_solution_length - key
        if savings >= min_savings:
            total += solutions[key]
        if debug:
            print(f"There are {solutions[key]} cheats that save {savings} picoseconds")

    if debug:
        print()

    return total


result = part_1(puzzle.example(0), 0, True)
display(result)
assert result == 44

###############
#...#...#.....#
#.#.#.#.#.###.#
#S#...#.#.#...#
#######.#.#.###
#######.#.#...#
#######.#.###.#
###..E#...#...#
###.#######.###
#...###...#...#
#.#####.#.###.#
#.#...#.#.#...#
#.#.#.#.#.#.###
#...#...#...###
###############

There are 14 cheats that save 2 picoseconds
There are 14 cheats that save 4 picoseconds
There are 2 cheats that save 6 picoseconds
There are 4 cheats that save 8 picoseconds
There are 2 cheats that save 10 picoseconds
There are 3 cheats that save 12 picoseconds
There are 1 cheats that save 20 picoseconds
There are 1 cheats that save 36 picoseconds
There are 1 cheats that save 38 picoseconds
There are 1 cheats that save 40 picoseconds
There are 1 cheats that save 64 picoseconds



44

In [4]:
# real case (part 1)
result = part_1(puzzle.input(), 100, False)
display(result)
assert result == 1497

1497

In [5]:
# test case (part 2)
def part_2(reader: InputReader, cheats: int, min_savings: int, debug: bool) -> int:
    matrix, start, end = domain_from_input(reader)
    initial_solution = find_shortest_path(matrix, start, end)

    solutions = {}

    def find_new_starts(point: tuple, max_distance: int) -> set:
        new_starts = set()
        for x in range(-max_distance, max_distance + 1):
            for y in range(-max_distance, max_distance + 1):
                new_point = (point[0] + x, point[1] + y)
                cost = abs(x) + abs(y)

                # ignore if cost is too high
                if max_distance < cost:
                    continue
                # ignore starting point
                if new_point == point:
                    continue
                # if in matrix
                if not matrix.pos_exists(point[0] + x, point[1] + y):
                    continue

                value = matrix.get_value(*new_point)

                # if within max distance
                if value != '#':
                    new_starts.add((new_point, cost))

        return new_starts

    for point in initial_solution:
        base_line = find_shortest_path_count(matrix, point, end)
        # find all "." within two steps
        new_starts = find_new_starts(point, cheats, )

        for new_start, new_steps in new_starts:
            # if there's a solution
            solution_length = find_shortest_path_count(matrix, new_start, end) + new_steps
            if 0 < solution_length < base_line:
                savings = base_line - solution_length
                solutions[savings] = solutions.get(savings, 0) + 1

    total = 0

    if debug:
        matrix.print()
        print()

    for savings in sorted(solutions.keys()):
        if savings >= min_savings:
            solution_count = solutions[savings]
            total += solution_count
            if debug:
                print(f"There are {solution_count} cheats that save {savings} picoseconds")

    if debug:
        print()

    return total


result = part_2(puzzle.example(0), 2, 0, True)
display(result)
assert result == 44

result = part_2(puzzle.example(0), 20, 50, True)
display(result)
assert result == 285

###############
#...#...#.....#
#.#.#.#.#.###.#
#S#...#.#.#...#
#######.#.#.###
#######.#.#...#
#######.#.###.#
###..E#...#...#
###.#######.###
#...###...#...#
#.#####.#.###.#
#.#...#.#.#...#
#.#.#.#.#.#.###
#...#...#...###
###############

There are 14 cheats that save 2 picoseconds
There are 14 cheats that save 4 picoseconds
There are 2 cheats that save 6 picoseconds
There are 4 cheats that save 8 picoseconds
There are 2 cheats that save 10 picoseconds
There are 3 cheats that save 12 picoseconds
There are 1 cheats that save 20 picoseconds
There are 1 cheats that save 36 picoseconds
There are 1 cheats that save 38 picoseconds
There are 1 cheats that save 40 picoseconds
There are 1 cheats that save 64 picoseconds



44

###############
#...#...#.....#
#.#.#.#.#.###.#
#S#...#.#.#...#
#######.#.#.###
#######.#.#...#
#######.#.###.#
###..E#...#...#
###.#######.###
#...###...#...#
#.#####.#.###.#
#.#...#.#.#...#
#.#.#.#.#.#.###
#...#...#...###
###############

There are 32 cheats that save 50 picoseconds
There are 31 cheats that save 52 picoseconds
There are 29 cheats that save 54 picoseconds
There are 39 cheats that save 56 picoseconds
There are 25 cheats that save 58 picoseconds
There are 23 cheats that save 60 picoseconds
There are 20 cheats that save 62 picoseconds
There are 19 cheats that save 64 picoseconds
There are 12 cheats that save 66 picoseconds
There are 14 cheats that save 68 picoseconds
There are 12 cheats that save 70 picoseconds
There are 22 cheats that save 72 picoseconds
There are 4 cheats that save 74 picoseconds
There are 3 cheats that save 76 picoseconds



285

In [6]:
# real case (part 2)
result = part_2(puzzle.input(), 20, 100, False)
display(result)
assert result == 1030809

1030809

In [7]:
# print easters eggs
puzzle.print_easter_eggs()

## Easter Eggs

<span title="If we give away enough mutexes, maybe someone will use one of them to fix the race condition!">winner</span> (If we give away enough mutexes, maybe someone will use one of them to fix the race condition!)